# Assessment 3 - Skeleton Code

In [ ]:
# load python libraries
import numpy as np
import pandas as pd
import re
import os
from bs4 import BeautifulSoup     # pulls data out of XML and HTML files
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
!pip install wget
import wget
import nltk

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=a418250aa31c468592a9577477cd730ec80ada8276ae088102f91ef07aae6c2e
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
# load the reviews from a csv file into Pandas
#data=pd.read_csv("AmazonReviews.csv",nrows=100000)
data=pd.read_csv('/content/AmazonReviews.zip')

FileNotFoundError: [Errno 2] No such file or directory: '/content/AmazonReviews.zip'

In [ ]:
# see how many entries there are
print(data['Text'].shape)

In [ ]:
# limit the data for this exercise otherwise it will take too long to train
# let's limit the data from 100,000 to 10,000
#data = data[:100000]
print(data['Text'].shape)

In [ ]:
data.head()

In [ ]:
# Display some of the data
data[['Score','Text','Summary']].sample(5)

In [ ]:
# Remove any duplicate entries in the data
data.drop_duplicates(subset=['Text'],inplace=True)
data.dropna(axis=0,inplace=True)

In [ ]:
labels = data['Score']
labels[labels<4] = 0
labels[labels>1] = 1

In [ ]:
# map words to colloquial terms
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [ ]:
# print out the first 10 entries in the data to view it to decide what to do for data cleaning
data['Text'][:10]

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')  # donload the stopwords library for use by NLTK
stop_words = set(stopwords.words('english'))

# create a function to clean the text.  Remove whiteapce, html characters, punctuation etc...
def text_cleaner(text):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    tokens = [w for w in newString.split() if not w in stop_words]
    long_words=[]
    for i in tokens:
        if len(i)>=3:                  #removing short word
            long_words.append(i)
    return (" ".join(long_words)).strip()

# store the cleaned text in a list
cleaned_text = []
for t in data['Text']:
    cleaned_text.append(text_cleaner(t))

In [ ]:
# create a function to clean the summary text
def summary_cleaner(text):
    newString = re.sub('"','', text)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = newString.lower()
    tokens=newString.split()
    newString=''
    for i in tokens:
        if len(i)>1:
            newString=newString+i+' '
    return newString

# Call the summary_cleaner function and store the cleaned summaries in a list
cleaned_summary = []
for t in data['Summary']:
    cleaned_summary.append(summary_cleaner(t))

# store the cleaned text and summaries in a dictionary
data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary
data['cleaned_summary'].replace('', np.nan, inplace=True)
# drop rows with null values
data.dropna(axis=0,inplace=True)

In [ ]:
# add the text _START_ and _END_ to the summaries to easily identify they are the summaries
data['cleaned_summary'] = data['cleaned_summary'].apply(lambda x : '_START_ '+ x + ' _END_')

In [ ]:
# print the first 5 reviews and their summaries
for i in range(5):
    print("Review:",data['cleaned_text'][i])
    print("Summary:",data['cleaned_summary'][i])
    print("Score:",data['Score'][i])
    print("\n")

In [ ]:
# investigate the distribution of the length of the text in the data
# to help fix the max length of the sequence
import matplotlib.pyplot as plt
text_word_count = []
summary_word_count = []

# populate the lists with sentence lengths
for i in data['cleaned_text']:
      text_word_count.append(len(i.split()))

for i in data['cleaned_summary']:
      summary_word_count.append(len(i.split()))

length_df = pd.DataFrame({'text':text_word_count, 'summary':summary_word_count})
length_df.hist(bins = 30)
plt.show()

In [ ]:
# from histograms, decide on a text review length of 80 and a summary length of 10
max_len_text=80
max_len_summary=10

# split the data into train and test sets
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(data['cleaned_text'],data['Score'],test_size=0.1,random_state=0,shuffle=True)

In [ ]:
#prepare a tokenizer for reviews on training data
max_words = 500    # the maximum number of words to keep, based on word frequency (size of the vocabulary)

x_tokenizer = Tokenizer(num_words=max_words, lower=True, split=' ')
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr    =   x_tokenizer.texts_to_sequences(x_tr)
x_val   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr,  maxlen=max_len_text, padding='post')
x_val   =   pad_sequences(x_val, maxlen=max_len_text, padding='post')

x_voc_size   =  len(x_tokenizer.word_index) +1

In [ ]:
print(x_tr[1])

In [ ]:
# Define the model
model = Sequential()
model.add(Embedding(x_voc_size, 300, input_length=80))  # Reduced embedding size
model.add(LSTM(500, return_sequences=True, recurrent_dropout=0.2))  # Added recurrent dropout
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))  # Added L2 regularization
model.add(Dropout(0.5)) # Added dropout layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model with a custom learning rate
optimizer = Adam(learning_rate=0.001)  # Adjusted learning rate
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Print model summary
model.summary()


In [ ]:
# Fit the model
model.fit(x_tr, y_tr, validation_data=(x_val, y_val), epochs=5, batch_size=128, verbose=1, callbacks=[early_stopping])

# Final evaluation of the model
scores = model.evaluate(x_val, y_val, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

The provided code snippets outline a comprehensive process for constructing, training, and assessing a neural network model tailored for sentiment analysis or text classification tasks. This methodology is vital for processing natural language data efficiently and accurately.
Data Preprocessing: The initial phase involves loading and preparing the dataset, including removing duplicates and missing values, cleaning the text to eliminate HTML tags, punctuation, and stopwords, and expanding contractions for uniformity in the text data (Kingma & Ba, 2015).
Data Exploration and Preparation: The dataset is explored to comprehend its structure and characteristics. This includes inspecting samples to glean insights into the text and summary content, as well as their associated scores. Additionally, the distribution of text and summary lengths is analyzed using histograms to determine suitable maximum sequence lengths (Prechelt, 1998).
Tokenization and Padding: Tokenization transforms the text data into numerical sequences suitable for input into the neural network model. The Keras Tokenizer class facilitates this process, and the resulting sequences are padded to ensure uniform length, aiding batch processing during training (Bottou, 2010).
Model Architecture: The neural network model is formulated using Keras' Sequential API, comprising embedding layers, LSTM layers, dropout layers, and dense layers. These layers are sequentially stacked to create a deep learning model capable of capturing intricate patterns in the input data. Regularization techniques, including dropout and L2 regularization, are integrated to mitigate overfitting (Sokolova & Lapalme, 2009).
Model Compilation: The model is compiled with appropriate loss function, optimizer, and evaluation metrics. Binary cross-entropy loss is selected for binary classification tasks, and the Adam optimizer is utilized with a custom learning rate. The model's performance is evaluated based on accuracy (Raschka & Mirjalili, 2019).
Model Training: The model is trained on the training dataset using the fit method. Through backpropagation, the model adjusts the weights of its layers to map input sequences to their corresponding output labels. Early stopping is employed to monitor validation loss and terminate training if no improvement is observed over a specified number of epochs (Raschka & Mirjalili, 2019).
Model Evaluation: Following training, the model is evaluated on the validation dataset to assess its generalization performance. The accuracy of the model on unseen data is calculated, providing insights into its classification capabilities and guiding further refinement if necessary (Bottou, 2010).
References
Kingma, D. P., & Ba, J. (2015). Adam: A Method for Stochastic Optimization. International Conference on Learning Representations.
Prechelt, L. (1998). Early Stopping - But When? In G. Tesauro, D. S. Touretzky, & T. K. Leen (Eds.), Advances in Neural Information Processing Systems 11 (pp. 55–61). MIT Press.
Bottou, L. (2010). Large-Scale Machine Learning with Stochastic Gradient Descent. Proceedings of COMPSTAT'2010.
Sokolova, M., & Lapalme, G. (2009). A Systematic Analysis of Performance Measures for Classification Tasks. Information Processing & Management, 45(4), 427–437.
Raschka, S., & Mirjalili, V. (2019). Python Machine Learning (3rd ed.). Packt Publishing.
